### Predicting Market crashes with topology in python
#### Beyond Traditional Indicators for Isights Into Market Behavior
https://medium.com/@crisvelasquez/predicting-stock-market-crashes-with-topological-data-analysis-in-python-1dc4f18ca7ca

[PH(D) = {[b,d]| b is birth time, d is death time of a feature in data D}]

In [9]:
%matplotlib inline

import numpy as np 
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
from matplotlib import style
import matplotlib.colors as mcolors
import yfinance as yf
from scipy.optimize import curve_fit
from ripser import ripser
from ripser import rips

# binance api 
import nasdaqdatalink
import quandl

quandl.ApiConfig.api_key = "AAHgDbrpouZ8KXGz3ihH"

#quandl.get("USTREASURY/REALYIELD")

start_date_INPUT = "2022-11-01"

BTC_recent = quandl.get("BCHAIN/MKPRU", start_date=start_date_INPUT)

plt.plot(BTC_recent.index, BTC_recent["Value"], color='k')

ImportError: cannot import name 'rips' from 'ripser' (/Users/parkjinhong/.pyenv/versions/3.10.3/lib/python3.10/site-packages/ripser/__init__.py)

In [2]:
log_returns = np.log(BTC_recent["Value"][1:]/BTC_recent["Value"][:-1])


In [8]:
window_size = 20
n = len(log_returns) - (2 * window_size) + 1
distances = np.full((n, 1), np.nan)  # Using np.full with NaN values

for i in range(n):
    segment1 = log_returns[i:i+window_size].reshape(-1, 1)
    segment2 = log_returns[i+window_size:i+(2*window_size)].reshape(-1, 1)

    if segment1.shape[0] != window_size or segment2.shape[0] != window_size:
        continue

    dgm1 = rips.fit_transform(segment1)
    dgm2 = rips.fit_transform(segment2)
    distance = persim.wasserstein(dgm1[0], dgm2[0], matching=False)
    distances[i] = distance



segment1 = log_returns[i:i + window_size].reshape(-1,1)

AttributeError: 'Series' object has no attribute 'reshape'